In [ ]:
%matplotlib inline

import numpy as np
from numpy.fft import fft2, ifft2, fftshift, ifftshift

from menpo.image import Image

# Network 

In [ ]:
from __future__ import division
import numpy as np
from numpy.fft import fft2, ifft2, ifftshift

from menpo.model import PCAModel, ICAModel, NMFModel
from menpo.math.decomposition.ica import _batch_ica, negentropy_logcosh
from menpo.image import Image
from menpo.visualize import print_dynamic, progress_bar_str

from alaborticcv2015.utils import pad, crop, multiconvsum, multiconvlist
from alaborticcv2015.deepconvkernel.generative import learn_pca_filters, learn_ica_filters 


class GenerativeDCK():

    def __init__(self, learn_filters=learn_pca_filters, n_levels=3,
                 n_filters=8, patch_size=(7, 7), mean_centre=False,
                 correlation=False, mode='same', boundary='constant'):
        self._learn_filters = learn_filters
        self.n_levels = n_levels
        self.n_filters = n_filters
        self.patch_size = patch_size
        self.mean_centre = mean_centre
        self.correlation = correlation
        self.mode = mode
        self.boundary = boundary

    def learn_network(self, images, group=None, label=None, verbose=False,
                      **kwargs):
        if verbose:
            string = '- Learning network'
        # initialize level_image and list of filters
        level_images = images
        self.filters = []
        for j in range(self.n_levels):
            if verbose:
                print_dynamic('{}: {}'.format(
                    string, progress_bar_str(j/self.n_levels, show_bar=True)))
            # extract level patches
            level_patches = self._extract_patches(level_images, group=group,
                                                  label=label)
            # learn level filters
            level_filters = self._learn_filters(level_patches, self.n_filters,
                                                **kwargs)
            # compute level responses lists
            level_images = self._compute_filter_responses(level_images,
                                                          level_filters)
            # save level filters
            self.filters.append(level_filters)
        if verbose:
            print_dynamic('{}: Done!\n'.format(string))

    def _extract_patches(self, images, group=None, label=None):
        patches = [i.extract_patches_around_landmarks(
                   group=group, label=label, patch_size=self.patch_size)
                   for i in images]
        return [p for ps in patches for p in ps]

    def _compute_filter_responses(self, images, filters):
        images = [multiconvlist(i, filters, mean_centre=self.mean_centre,
                                correlation=self.correlation, mode=self.mode,
                                boundary=self.boundary) for i in images]
        return [i for imgs in images for i in imgs]

    def learn_kernel(self, level=None, ext_shape=None):
        kernel = 1
        for level_filters in self.filters[:level]:
            k = 0
            for f in level_filters:
                if ext_shape is not None:
                    f_pixels = pad(f.pixels, ext_shape)
                else:
                    f_pixels = f.pixels
                fft_f = fft2(f_pixels)
                k += fft_f.conj() * fft_f
            kernel *= k
        return Image(kernel)

    def compute_network_response(self, image, level=None):
        images = [image]
        for level_filters in self.filters[:level]:
            images = self._compute_filter_responses(images, level_filters)
        return self._list_to_image(images)

    @classmethod
    def _list_to_image(cls, images):
        img = images[0]
        n_ch, h, w = img.pixels.shape
        pixels = np.zeros((len(images) * n_ch, h, w))
        for j, i in enumerate(images):
            ind = j * n_ch
            pixels[ind:ind+n_ch] = i.pixels
        img = Image(pixels)
        img.landmarks = images[0].landmarks
        return img

    def compute_kernel_response(self, i, level=None):
        # extended shape
        i_shape = np.asarray(i.shape)
        f_shape = np.asarray(self.filters[0][0].shape)
        ext_shape = i_shape + f_shape - 1

        # extend image and filter
        ext_i = pad(i.pixels, ext_shape, mode=self.boundary)

        # compute ffts of extended image and extended filter
        fft_ext_i = fft2(ext_i)
        # compute deep convolutional kernel
        fft_ext_f = self.learn_kernel(level=level, ext_shape=ext_shape).pixels

        # compute extended convolution in Fourier domain
        fft_ext_c = fft_ext_f**0.5 * fft_ext_i

        # compute ifft of extended convolution
        ext_c =  np.real(ifft2(fft_ext_c))

        if self.mode is 'full': 
            c = Image(ext_c)
            c.landmarks = i.landmarks
            for key in c.landmarks.keys():
                c.landmarks[key].lms.points += (f_shape - 1) // 2  
        elif self.mode is 'same':
            c = Image(crop(ext_c, i_shape))
            c.landmarks = i.landmarks
        elif self.mode is 'valid':
            c = Image(crop(ext_c, i_shape - f_shape + 1))
            c.landmarks = i.landmarks
            for key in c.landmarks.keys():
                c.landmarks[key].lms.points -= (f_shape - 1) // 2  
        else:
            raise ValueError(
                "mode={}, is not supported. The only supported "
                "modes are: 'full', 'same' and 'valid'.".format(mode)) 

        return c

In [ ]:
from __future__ import division
import numpy as np
from numpy.fft import fft2, ifft2, ifftshift

from menpo.model import PCAModel, ICAModel, NMFModel
from menpo.math.decomposition.ica import _batch_ica, negentropy_logcosh
from menpo.image import Image
from menpo.visualize import print_dynamic, progress_bar_str

from alaborticcv2015.utils import pad, crop, multiconvsum, multiconvlist
from alaborticcv2015.deepconvkernel.generative import learn_pca_filters, learn_ica_filters 


class GenerativeDCK():

    def __init__(self, learn_filters=learn_pca_filters, n_levels=3,
                 n_filters=8, patch_size=(7, 7), mean_centre=False,
                 correlation=False, mode='same', boundary='constant'):
        self._learn_filters = learn_filters
        self.n_levels = n_levels
        self.n_filters = n_filters
        self.patch_size = patch_size
        self.mean_centre = mean_centre
        self.correlation = correlation
        self.mode = mode
        self.boundary = boundary

    def learn_network(self, images, group=None, label=None, verbose=False,
                      **kwargs):
        if verbose:
            string = '- Learning network'
        # initialize level_image and list of filters
        level_images = images
        self.filters = []
        for j in range(self.n_levels):
            if verbose:
                print_dynamic('{}: {}'.format(
                    string, progress_bar_str(j/self.n_levels, show_bar=True)))
            # extract level patches
            level_patches = self._extract_patches(level_images, group=group,
                                                  label=label)
            # learn level filters
            level_filters = self._learn_filters(level_patches, self.n_filters,
                                                **kwargs)
            # compute level responses lists
            level_images = self._compute_filter_responses(level_images,
                                                          level_filters)
            # save level filters
            self.filters.append(level_filters)
        if verbose:
            print_dynamic('{}: Done!\n'.format(string))

    def _extract_patches(self, images, group=None, label=None):
        patches = [i.extract_patches_around_landmarks(
                   group=group, label=label, patch_size=self.patch_size)
                   for i in images]
        return [p for ps in patches for p in ps]

    def _compute_filter_responses(self, images, filters):
        images = [multiconvlist(i, filters, mean_centre=self.mean_centre,
                                correlation=self.correlation, mode=self.mode,
                                boundary=self.boundary) for i in images]
        return [i for imgs in images for i in imgs]

    def learn_kernel(self, level=None):
        kernel = 1
        for level_filters in self.filters[:level]:
            k = 0
            for f in level_filters:
                fft_f = fft2(f.pixels)
                k += fft_f.conj() * fft_f
            kernel *= k
        return Image(kernel)

    def compute_network_response(self, image, level=None):
        images = [image]
        for level_filters in self.filters[:level]:
            images = self._compute_filter_responses(images, level_filters)
        return self._list_to_image(images)

    @classmethod
    def _list_to_image(cls, images):
        img = images[0]
        n_ch, h, w = img.pixels.shape
        pixels = np.zeros((len(images) * n_ch, h, w))
        for j, i in enumerate(images):
            ind = j * n_ch
            pixels[ind:ind+n_ch] = i.pixels
        img = Image(pixels)
        img.landmarks = images[0].landmarks
        return img

    def compute_kernel_response(self, i, level=None):
        # extended shape
        i_shape = np.asarray(i.shape)
        f_shape = np.asarray(self.filters[0][0].shape)
        ext_shape = i_shape + f_shape - 1

        # extend image and filter
        ext_i = pad(i.pixels, ext_shape, mode=self.boundary)

        # compute ffts of extended image and extended filter
        fft_ext_i = fft2(ext_i)
        # compute deep convolutional kernel
        fft_ext_f = self.learn_kernel(level=level, ext_shape=ext_shape).pixels

        # compute extended convolution in Fourier domain
        fft_ext_c = fft_ext_f**0.5 * fft_ext_i

        # compute ifft of extended convolution
        ext_c =  np.real(ifft2(fft_ext_c))

        if self.mode is 'full': 
            c = Image(ext_c)
            c.landmarks = i.landmarks
            for key in c.landmarks.keys():
                c.landmarks[key].lms.points += (f_shape - 1) // 2  
        elif self.mode is 'same':
            c = Image(crop(ext_c, i_shape))
            c.landmarks = i.landmarks
        elif self.mode is 'valid':
            c = Image(crop(ext_c, i_shape - f_shape + 1))
            c.landmarks = i.landmarks
            for key in c.landmarks.keys():
                c.landmarks[key].lms.points -= (f_shape - 1) // 2  
        else:
            raise ValueError(
                "mode={}, is not supported. The only supported "
                "modes are: 'full', 'same' and 'valid'.".format(mode)) 

        return c

# Test 

### Load images

In [ ]:
import menpo.io as mio
from menpo.landmark import labeller, ibug_face_66

training_images = []
for i in mio.import_images('/data/PhD/DataBases/faces/lfpw/trainset/', verbose=True, 
                           max_images=25):
    
    i.crop_to_landmarks_proportion_inplace(1)
    i = i.rescale_landmarks_to_diagonal_range(100)
    labeller(i, 'PTS', ibug_face_66)
    if i.n_channels == 3:
        i = i.as_greyscale(mode='average')
    training_images.append(i)

In [ ]:
from menpo.visualize import visualize_images

visualize_images(training_images)

# Deep Convolutional Kernels

In [ ]:
dck = GenerativeDCK(learn_filters=learn_pca_filters, n_filters=8,
                    patch_size=(7, 7), n_levels=2, mean_centre=False,
                    correlation=False, mode='full', boundary='constant')

In [ ]:
dck.learn_network(training_images, group='ibug_face_66', mean_centre=True, verbose=True)

In [ ]:
visualize_images(dck.filters[-1])

In [ ]:
k = Image(np.real(fftshift(ifft2(dck.learn_kernel(level=None).pixels))))
k.view()

In [ ]:
def conv(i, f, mean_centre=False, correlation=False, mode='same', 
         boundary='constant'):
    if mean_centre:
        i = i.copy()
        f = f.copy()
        i.mean_centre_inplace()
        f.mean_centre_inplace()
        
    if correlation:
        f.pixels = f.pixels[:, ::-1, ::-1]
    
    # extended shape
    i_shape = np.asarray(i.shape)
    f_shape = np.asarray(f.shape)
    ext_shape = i_shape + f_shape - 1

    # extend image and filter
    ext_i = pad(i.pixels, ext_shape, mode=boundary)
    ext_f = pad(f.pixels, ext_shape)

    # compute ffts of extended image and extended filter
    fft_ext_i = fft2(ext_i)
    fft_ext_f = fft2(ext_f)

    # compute extended convolution in Fourier domain
    fft_ext_c = fft_ext_f * fft_ext_i

    # compute ifft of extended convolution
    ext_c = ifftshift(ifft2(fft_ext_c), axes=(-2, -1))

    if mode is 'full': 
        c = Image(ext_c)
        c.landmarks = i.landmarks
        for key in c.landmarks.keys():
            c.landmarks[key].lms.points += (f_shape - 1) // 2  
    elif mode is 'same':
        c = Image(crop(ext_c, i_shape))
        c.landmarks = i.landmarks
    elif mode is 'valid':
        c = Image(crop(ext_c, i_shape - f_shape + 1))
        c.landmarks = i.landmarks
        for key in c.landmarks.keys():
            c.landmarks[key].lms.points -= (f_shape - 1) // 2  
    else:
        raise ValueError(
            "mode={}, is not supported. The only supported "
            "modes are: 'full', 'same' and 'valid'.".format(mode)) 
        
    return c

In [ ]:
from menpo.feature import imgfeature

@imgfeature
def deep_kernel_features(img, level=None):
    feature = dck.compute_kernel_response(img, level=level)
    return feature

@imgfeature
def deep_kernel_features2(img, level=None):
    feature = Image(conv(img, k, mode='same').pixels)
    feature.landmarks = img.landmarks
    return feature

@imgfeature
def deep_network_features(img, level=None):
    feature = dck.compute_network_response(img, level=level)
    return feature

In [ ]:
visualize_images([deep_network_features(i, level=None) for i in training_images[:10]])

In [ ]:
visualize_images([Image(np.abs(deep_kernel_features2(i, level=None).pixels)) 
                  for i in training_images[:10]])

In [ ]:
from menpofit.aam.base import build_reference_frame
from menpo.shape import mean_pointcloud
from menpo.transform import PiecewiseAffine

# build reference frame
shapes = [i.landmarks['PTS'].lms for i in training_images]
mean_shape = mean_pointcloud(shapes)
reference_frame = build_reference_frame(mean_shape)

# warp images
transforms = [PiecewiseAffine(reference_frame.landmarks['source'].lms, s) for s in shapes]
warped_images = [i.warp_to_mask(reference_frame.mask, t) for (i, t) in zip(training_images, transforms)]

img1 = warped_images[1]
img2 = warped_images[2]

print img1.shape
print img2.shape

In [ ]:
shape = img1.shape

v1n = crop(deep_network_features(Image(img1.pixels)).pixels, shape)
v2n = crop(deep_network_features(Image(img2.pixels)).pixels, shape)
dotn = v1n.ravel().T.dot(v2n.ravel()) 

vn = crop(deep_network_features(Image(img1.pixels - img2.pixels)).pixels, shape)
l2n = vn.ravel().T.dot(vn.ravel())

v1k = deep_kernel_features2(Image(img1.pixels)).pixels
v2k = deep_kernel_features2(Image(img2.pixels)).pixels
dotk = v1k.ravel().conj().T.dot(v2k.ravel()) 

vk = deep_kernel_features2(Image(img1.pixels - img2.pixels)).pixels
l2k = vk.ravel().conj().T.dot(vk.ravel())

print 'Dot Ratio:', dotn / dotk
print 'L2 Ratio:', l2n / l2k

In [ ]:
print dotn
print dotk

print l2n
print l2k

In [ ]:
import menpo.io as mio
from menpo.landmark import labeller, ibug_face_49, ibug_face_66_trimesh

test_images = []
for i in mio.import_images('/data/PhD/DataBases/faces/afw/', verbose=True, 
                           max_images=10):
    
    i.crop_to_landmarks_proportion_inplace(1)
    i = i.rescale_landmarks_to_diagonal_range(100)
    labeller(i, 'PTS', ibug_face_49)
    labeller(i, 'PTS', ibug_face_66)
    labeller(i, 'PTS', ibug_face_66_trimesh)
    if i.n_channels == 3:
        i = i.as_greyscale(mode='average')
    test_images.append(i)

In [ ]:
from menpo.feature import no_op, fast_dsift
from alabortijcv2015.aam import GlobalAAMBuilder

builder = GlobalAAMBuilder(features=deep_kernel_features2, diagonal=100, 
                           scale_shapes=False, scales=(1, .5))
aam = builder.build(training_images, group='ibug_face_66', verbose=True)

In [ ]:
from menpofit.visualize import visualize_appearance_model

visualize_appearance_model(aam.appearance_models[-1])

In [ ]:
from alabortijcv2015.aam import StandardAAMFitter
from alabortijcv2015.aam.algorithm import SIC, AIC, MAIC
                  
fitter = StandardAAMFitter(aam, algorithm_cls=AIC, n_shape=[3, 6], 
                           n_appearance=[100, 100], sampling_step=4)

In [ ]:
np.random.seed(seed=1)

fitter_results = []

for j, i in enumerate(test_images[:10]):
    
    gt_s = i.landmarks['ibug_face_66'].lms
    s = fitter.perturb_shape(gt_s, noise_std=0.04)
    
    fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=20, map_inference=False)
    fr.downscale = 0.5
    
    fitter_results.append(fr)
    
    print 'Image: ', j
    print fr

In [ ]:
np.random.seed(seed=1)

fitter_results = []

for j, i in enumerate(test_images[:10]):
    
    gt_s = i.landmarks['ibug_face_66'].lms
    s = fitter.perturb_shape(gt_s, noise_std=0.04)
    
    fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=20, map_inference=False)
    fr.downscale = 0.5
    
    fitter_results.append(fr)
    
    print 'Image: ', j
    print fr

In [ ]:
from menpofit.visualize import visualize_fitting_results
    
visualize_fitting_results(fitter_results)